In [243]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import string
from collections import Counter, defaultdict
import re

In [244]:
data_path = r"C:\Projects\connecteddatahub\data"
years = ["1999","2000","2005","2007","2008","2009","2010","2011","2013","2018",]
# years = ['2013']

In [245]:
year = '2010'
full_path = os.path.join(data_path, 'gpt_dataframes', f'{year}_gptDataframe.csv')
grouped_path = os.path.join(data_path, 'cleaned_dataframes', 'subinstitutions', f'{year}_grouped.csv')
grouped_validated_path = os.path.join(data_path, 'cleaned_dataframes', 'subinstitutions', 'full_validation.csv')

In [246]:
# all_dfs = []
# for year_ in years:
#     # build the path for this year’s grouped file
#     grouped_path = os.path.join(
#         data_path,
#         'cleaned_dataframes',
#         'subinstitutions',
#         f'{year_}_grouped.csv'
#     )
#     # read and collect
#     df = pd.read_csv(grouped_path)
#     all_dfs.append(df)

# # concatenate, drop duplicates across all columns
# full_df = pd.concat(all_dfs, ignore_index=True).drop_duplicates()

# # write out the merged, deduped DataFrame
# full_df.to_csv(grouped_validated_path, index=False)


In [247]:
fulldf = pd.read_csv(full_path)

In [248]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation", "applied science", "computational science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction", "school of construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership", "policy science"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home", "family studies"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome", "occupational therapy"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions", "professional campus"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": []
}

In [249]:
keyword2category = {k:c for c, ks in keywords.items() for k in ks}
#dictionary of "school of" + category and #college of" + category
schoolwords = {"school of "+s.lower():s for s in keywords.keys()}
schoolwords.update({"college of "+s.lower():s for s in keywords.keys()})
schoolwords['school of medicine'] = "Medical"
schoolwords['school of management'] = "Business"
schoolwords['college of management'] = "Business"
schoolwords['school of economics'] = "Business"
schoolwords['teachers college'] = "Education"
schoolwords["college of science"] = "Natural Sciences"
schoolwords["school of science"] = "Natural Sciences"
schoolwords["college of sciences"] = "Natural Sciences"
schoolwords["school of sciences"] = "Natural Sciences"
schoolwords["college of design"] = "Fine Arts"
schoolwords["school of design"] = "Fine Arts"
schoolwords["college of art"] = "Fine Arts"
schoolwords["school of art"] = "Fine Arts"
schoolwords["college of arts"] = "Fine Arts"
schoolwords["school of arts"] = "Fine Arts"
schoolwords["library"] = "Administration"
schoolwords["libraries"] = "Administration"
schoolwords["students"]  = "Administration"
schoolwords["university library"] = "Administration"

In [250]:
#create table of unique institution + subinstitutions
unique_combinations = set()
all_subinstitutions = []
for index, row in fulldf.iterrows():
    institution = row["Institution"]
    subinstitution = row["SubInstitution"]
    id = row["AffiliationId"]
    combination = (institution, subinstitution, id)
    if combination not in unique_combinations and pd.notnull(subinstitution):
        unique_combinations.add(combination)
        all_subinstitutions.append(combination)

unique_df = pd.DataFrame(all_subinstitutions, columns=["Institution", "SubInstitution", "AffiliationId"])
unique_df["Category"] = ""
print(unique_df.shape)

(4608, 4)


In [251]:
#now find the high-confidence classifcations. e.g. "school of + category"
#Checking for equality, not substring
unique_df["Category"] = ""
high_confidence = pd.DataFrame(columns=["Institution", "SubInstitution", "Category", "AffiliationId"])
uncategorized_df = pd.DataFrame(columns=["Institution", "SubInstitution", "Category", "AffiliationId"])

for index, row in unique_df.iterrows():
    subinstitution = row["SubInstitution"].lower()
    for key, value in schoolwords.items():
        if key == subinstitution:
            if(row["Category"] == ""):
                unique_df.at[index, "Category"] += str(value)
                
            else:
                unique_df.at[index, "Category"] += str(value) + ", "
    if unique_df.at[index, "Category"] != "":
        high_confidence = pd.concat([high_confidence, unique_df.loc[[index]]], ignore_index=True)
    else:
        uncategorized_df = pd.concat([uncategorized_df, unique_df.loc[[index]]], ignore_index=True)
        
print(high_confidence.shape)


C:\Users\tykun\AppData\Local\Temp\ipykernel_23480\3546724983.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  uncategorized_df = pd.concat([uncategorized_df, unique_df.loc[[index]]], ignore_index=True)
C:\Users\tykun\AppData\Local\Temp\ipykernel_23480\3546724983.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  high_confidence = pd.concat([high_confidence, unique_df.loc[[index]]], ignore_index=True)


(1426, 4)


In [252]:
#categorize rest of the subinstitutions, including those who fall under multiple categories
medium_confidence = pd.DataFrame(columns=["Institution", "SubInstitution", "Category", "AffiliationId"])
uncategorized_df["Category"] = ""

for index, row in uncategorized_df.iterrows():
    institution = row["Institution"].title()
    subinstitution = row["SubInstitution"].title()
    #check every word in the subinstitution
    matched = False
    for key, value in keyword2category.items():
        # print(word, key)
        if key.title() in subinstitution:
            if not matched:
                uncategorized_df.at[index, "Category"] += str(value)
                matched = True
            elif matched and str(value) not in uncategorized_df.at[index, "Category"]:
                uncategorized_df.at[index, "Category"] += ", " + str(value)
    if uncategorized_df.at[index, "Category"] != "":
        medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)
        unique_df.loc[unique_df['SubInstitution'].str.title() == subinstitution, 'Category'] = uncategorized_df.at[index, 'Category']
    elif institution in subinstitution and uncategorized_df.at[index, "Category"] == "" and institution != subinstitution:
        uncategorized_df.at[index, "Category"] = "Satellite Campus"
        medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)
        unique_df.loc[unique_df['SubInstitution'].str.title() == subinstitution, 'Category'] = uncategorized_df.at[index, 'Category']

C:\Users\tykun\AppData\Local\Temp\ipykernel_23480\3016885815.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)


In [253]:
#cleanup incorrect classifications
for index, row in medium_confidence.iterrows():
    category = row["Category"]
    subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    art_count = word_list.count("Arts")
    engineering_count = word_list.count("Engineering")

    if "Humanities" in category and "Arts and Sciences" in category and "Fine Arts" in category and "Fine Arts" in subinstitution and art_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Arts and Sciences", "").strip(", ")
    if "Communication Arts And Science" in subinstitution and "Arts and Sciences" in category:
        medium_confidence.at[index, "Category"] = category.replace("Arts and Sciences", "").strip(", ")
    if "Arts and Sciences" not in category and "Fine Arts" not in category and "Liberal Art" not in subinstitution and art_count == 1:
         medium_confidence.at[index, "Category"] += ", Fine Arts"
    if "Administration" in category:
        medium_confidence.at[index, "Category"] = "Administration"
    if "Arts," in subinstitution and "Arts And Sciences" not in category and "Liberal Arts" not in category and "Fine Arts" not in category:
        medium_confidence.at[index, "Category"] += ", Fine Arts"
    if "And Science" in subinstitution and "Arts and Sciences" not in category and "Natural Sciences" not in category:
        medium_confidence.at[index, "Category"] += ", Natural Sciences"
    if "Liberal Arts" in category and "Arts and Sciences" in category:
        medium_confidence.at[index, "Category"] = category.replace("Liberal Arts", "").strip(", ")
    if "Software Engineering" in subinstitution and "Engineering" in category and engineering_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Engineering", "").strip(", ")
    if "Hospitality" in subinstitution and "Hospital" in category:
        medium_confidence.at[index, "Category"] = category.replace("Hospital,", "").strip(", ")
    if "Foundation School" in subinstitution:
        medium_confidence.at[index, "Category"] = category.replace("Foundation", "").strip(", ")

In [254]:
#categorize remaining "others"

uncategorized_df["Category"] = ""
na_df = unique_df[unique_df["Category"] == ""].copy()
na_df["Category"] = "Other"
    
institution_list = list(set(fulldf["Institution"].tolist()))

for index, row in na_df.iterrows():
    category = row["Category"]
    subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    institution_appears = any(substring in subinstitution for substring in institution_list)
    #remaining schools with "library" are not library sciences/information science anymore
    if "Library" in subinstitution:
        na_df.at[index, "Category"] = "Administration"

In [255]:
combination_counter = Counter()
subinstitution_texts = {}

for index, row in medium_confidence.iterrows():
    category_list = row["Category"]
    subinstitution = row["SubInstitution"]
    categories = sorted([cat.strip() for cat in category_list.split(",") if cat.strip()])

    if len(categories) > 1:
        for size in range(2, len(categories) + 1):
            for c in combinations(categories, size):
                #convert combination to a tuple to use it as dict key
                c_tuple = tuple(c)
                combination_counter[c_tuple] += 1
                
                if c_tuple not in subinstitution_texts:
                    subinstitution_texts[c_tuple] = []
                subinstitution_texts[c_tuple].append(subinstitution)

sorted_combinations = sorted(combination_counter.items(), key=lambda item: item[1], reverse=True)

#Displays combinations incorrectly e.g.Category = "Business, Humanaties, Social Sciences" will not only display as that combination,
#but also display as "Business, humanities" and "Business, Social Sciences", etc, so don't use
#to gauge accuracy of classification
print("Category combinations:")
for combination, count in sorted_combinations:
    print(f"{combination}: {count}")
    print("SubInstitutions:", subinstitution_texts[combination])
    print()

Category combinations:
('Fine Arts', 'Humanities'): 47
SubInstitutions: ['College Of Humanities And Fine Arts', 'School Of Fine Arts And Humanities', 'College Of Humanities And Fine Arts', 'College Of Arts And Humanities', 'College Of Arts And Humanities', 'College Of Arts Humanities And Social Sciences', 'College Of Arts Humanities And Social Sciences', 'College Of Humanities And Arts', 'School Of Arts And Humanities', 'College Of Arts And Humanities', 'School Of Arts And Humanities', 'College Of Architecture Arts And Humanities', 'College Of Arts And Humanities', 'School Of Arts Humanities And Social Sciences', 'College Of Arts And Humanities', 'School Of Humanities Arts And Social Sciences [Shass]', 'Residential College In Arts And Humanities', 'College Of Arts And Humanities', 'College Of Arts And Humanities', 'College Of Humanities And Fine Arts', 'College Of Arts Humanities And Social Sciences', 'College Of Humanities Fine And Performing Arts', 'School Of Arts And Humanities', 'A

In [256]:
# Combine the DataFrames
combined_df = pd.concat([medium_confidence, na_df, high_confidence], ignore_index=True)
sorted_combined_df = combined_df.sort_values(by="Category")

In [257]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    admin_list = ["Services", "Programs", "Resources", "Freshman", "Freshmen", "Association", "Plans", "Planning", "Outreach", "Faculty"]
    if any(s in subinstitution for s in admin_list):
        sorted_combined_df.at[index, "Category"] = "Administration"
sorted_combined_df = combined_df.sort_values(by="Category")

In [258]:
category_counter = Counter()
for category_list in combined_df["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

sorted_category = dict(sorted(category_counter.items(), key=lambda item: item[1], reverse=True))
print("\nCategory counts")
for category, number in sorted_category.items():
    print(f"{category}: {number}")


Category counts
Business: 484
Education: 403
Graduate: 372
Arts and Sciences: 315
Engineering: 280
Health Science: 273
Administration: 252
Fine Arts: 252
Other: 234
Natural Sciences: 229
Foundation: 218
Nursing: 187
Law: 159
Medical: 141
Social Work: 133
Technology: 105
Humanities: 102
Social Sciences: 98
Liberal Arts: 98
Behavioral Science: 84
Policy and Affairs: 80
Communication: 79
Pharmacy: 77
Agriculture: 74
Continued Studies: 74
Archictecure: 73
Environmental and Earth Science: 70
Professional Studies: 59
Information: 56
Dentistry: 54
Honor College: 41
Religious Studies: 40
Computer Science: 40
Satellite Campus: 38
University College: 34
Journalism: 30
Veterinary: 28
Physical Therapy and Nutrition: 14
Language and Culture: 12
Optometry: 11
Hospitality: 10
Marine Science: 10
Family and Consumer Science: 7
Criminal Justice: 6
Hospital: 3
Pathology: 2
Boards: 1
Audiology: 1
Culinary: 1
Gerontology: 1


In [259]:
validated_df = pd.read_csv(grouped_validated_path)
validated_df.sort_values(by=["Category", "Institution"])
validated_subinstitution_list = list(zip(validated_df["Institution"].tolist(), validated_df["SubInstitution"].tolist()))
validated_subinstitution_dict = {key: value for key, value in zip(validated_subinstitution_list, validated_df["Category"].tolist())}

#categorize remaining others with the validation list
for index, row in combined_df.iterrows():
    inst_sub = (row["Institution"], row["SubInstitution"])
    if row["Category"] == "Other" and inst_sub in validated_subinstitution_dict.keys():
        print(inst_sub, "    ", validated_subinstitution_dict[inst_sub]) 
        combined_df.at[index, "Category"] = validated_subinstitution_dict[inst_sub]
    elif row["SubInstitution"] == "College":
        combined_df.at[index, "Category"] = "Error"
#number of elements that have yet to be categorized
sorted_combined_df = combined_df.sort_values(by=["Category", "Institution"])
category_counter = Counter(sorted_combined_df['Category'])
num_other = category_counter['Other']
total = len(sorted_combined_df)
print("\n\n Remaining unclassified: ", 100*(num_other/total), "%")

('Agnes Scott College', 'College')      Error
('Alliant International University', 'School Of Forensic Studies')      Other
('Arizona State University', 'College Of Public Programs')      Other
('Arizona State University', 'Katherine K Herberger College Of Arts')      Other
('Bard College', 'Forest City Ratner Companies')      Other
('Bates College', 'Faculty')      Other
('Birmingham Southern College', 'First Year Programs')      Other
('University Of Iowa', 'International Programs')      Administration
('University Of Iowa', 'University Of Lowa Health Care')      Other
('Boston University', 'Metropolitan College')      Continued Studies
('Bowdoin College', 'Osterweis Capital Management Inc')      Other
('Brown University', 'Undergraduate College')      Other
('Bucknell University', 'Summer School')      Other
('California Polytechnic State University San Luis Obispo', 'Cal Poly Corporation')      Other
('California State University Fullerton', 'Auxiliary Services Corporation')      O

In [260]:
#final df
sorted_combined_df.to_csv(grouped_path, index=False)

category_counter = Counter()
for category_list in combined_df["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

sorted_category = dict(sorted(category_counter.items(), key=lambda item: item[1], reverse=True))
print("\nFinal category counts")
for category, number in sorted_category.items():
    print(f"{category}: {number}")


Final category counts
Business: 486
Education: 405
Graduate: 372
Arts and Sciences: 315
Engineering: 280
Health Science: 273
Administration: 270
Fine Arts: 254
Natural Sciences: 229
Foundation: 218
Nursing: 187
Other: 172
Law: 160
Medical: 142
Social Work: 133
Technology: 105
Humanities: 103
Liberal Arts: 103
Social Sciences: 99
Behavioral Science: 84
Policy and Affairs: 80
Communication: 79
Continued Studies: 79
Pharmacy: 77
Agriculture: 74
Archictecure: 73
Environmental and Earth Science: 70
Professional Studies: 60
Information: 56
Dentistry: 54
Honor College: 42
Satellite Campus: 42
University College: 42
Religious Studies: 40
Computer Science: 40
Journalism: 30
Veterinary: 28
Physical Therapy and Nutrition: 14
Language and Culture: 12
Error: 12
Optometry: 11
Hospitality: 10
Marine Science: 10
Family and Consumer Science: 7
Criminal Justice: 6
Hospital: 3
Pathology: 2
Boards: 1
Audiology: 1
Culinary: 1
Gerontology: 1
